<a href="https://colab.research.google.com/github/monta0315/pytorch_pra/blob/main/saveloadrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#モデルの保存と読み込み
モデルの状態を継続させるために、モデルを保存する方法とモデルを読み込む推論を実行する方法について解説する

In [1]:
%matplotlib inline

In [3]:
import torch
import torch.onnx as onnx
import torchvision.models as models

#モデルの重みの保存と読み込み
Pytorchのモデルは学習したパラメータを内部に辞書型で保持している

これらのパラメータの値はtorch.saveを使用することで、永続化させることができる



In [8]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(),'models_weights.pth')

モデルの重みを読み込むためには、予め、同じモデルの形をしたインスタンスを用意する

そしてそのインスタンスに対してload_state_dict()メソッドを使用しパラメータの値を読み込む

In [10]:
model = models.vgg16()
model.load_state_dict(torch.load('models_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

ドロップアウトやバッチノーマライゼーションレイヤーをevaluationモードに切り替えるために、推論前にはmodel.eval()を実行する

これを忘れると推論結果が正しくなくなる

#モデルの形ごと保存、読み込む方法

モデルの重みをロードする場合は、先のモデルのインスタンスを用意する必要がある

モデルクラスの構造も一緒に保存したい場合は保存時にmodel.state_dict()ではなくmodelに渡す


In [11]:
#torch.saveの第一引数がmodel.state_dictではなくmodelに変換されている
torch.save(model,'model.pth')

In [13]:
#モデルのをロードする
model = torch.load('model.pth')

【注意】

上記の方法はPythonの[`pickle`](https://docs.python.org/3/library/pickle.html)モジュールをモデルのシリアライズに使用します。

そのため、モデルのロード時に実際のクラス定義が利用可能である必要があります。

<br>

【日本語訳注】

上記の表現は理解が少し難しいのですが、言いたいことは、モデルのモジュールに独自クラスを定義して使用している場合、`torch.load`を実行する前に、その独自クラスをimportするか宣言するかして、使用可能な状態にしておく必要があります、という意味です。

でないと、`load`時に不明なクラスを使用することになり読み込みエラーとなります。



#ONNX形式でもモデル出力

PytorchはONNX形式でのモデル出力もサポートしている

しかし、Pytorchの計算グラフは動的に生成されるため、出力処理では計算グラフを一度実行して作成してからONNXモデルを生成する必要がある

すなわち実際に一度データを流してみる必要がある

そのため、テスト用の適切なテンソルサイズの入力変数を用意し、モデル出力の処理に渡す必要がある

以下ではダミーのゼロテンソルを適切なサイズで作成して使用している

In [14]:
input_image = torch.zeros(1,3,224,224)
onnx.export(model,input_image,'model.onnx')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
